<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">TFM - Previsión de demanda mediante uso de técnicas de machine learning
</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Carlos Pérez Cebrián</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Combinación Clasificación - Regresión

Como el número de días en los que no se registran ventas (udsVenta=0) es muy significativo, vamos a dividir el ejercicio en un problema de clasificación y un problema de regresión.

El problema de clasificación identificará si ese día hay ventas o no, para ello usaremos el modelo XGBClassifier.
En una segunda fase se aplica el modelo de regresión para predecir la cantidad de ventas solo cuando haya ventas (XGBRegressor).
El modelo de regresión debe predecir valores 0 cuando el clasificador indentifique que ese día no habrá ventas.

Decidimos entrenar el modelo de regresión con todo el conjunto de datos, y usar el clasificador para ajustar los días sin ventas a 0. 

XGBoost (Extreme Gradient Boosting) es una versión optimizada y escalable del algoritmo de gradient boosting, diseñada para velocidad, precisión y eficiencia. Combina múltiples modelos débiles, típicamente árboles de decisión, para crear un modelo más fuerte. XGBoost se destaca por su eficiencia computacional y rendimiento.

Es ideal para tareas que requieren grandes volúmenes de datos y alta precisión, especialmente en clasificación y regresión, utilizando ensemble learning y gradient boosting para minimizar errores.

Ensemble learning: combinación de múltiples modelos para obtener un modelo más robusto y preciso. La idea es que al combinar varios modelos, se pueden compensar los errores individuales de cada uno. 

Boosting: Se entrenan modelos secuencialmente, donde cada modelo intenta corregir los errores del modelo anterior. 

Gradient boosting es una técnica de boosting que se enfoca en minimizar los errores de predicción mediante la optimización del gradiente. Imagina que tienes un conjunto de datos y un modelo inicial que comete ciertos errores. Con cada iteración de gradient boosting, se añade un nuevo modelo que corrige esos errores, reduciendo progresivamente la cantidad de error total.


Cargamos las siguientes librerías necesarias:

In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBClassifier, XGBRegressor
import math

Cargar el conjunto de datos

(Nota: el conjunto de datos es el resultado de una fase de exploración, analisis y preparación previa.)

In [2]:
# Cargar el archivo CSV con los datos de ventas
file_path = "Data/VentasRdo.csv"
file_path = "Data/VentasRdo_10productos.csv"
df = pd.read_csv(file_path, parse_dates=['idSecuencia'])

# Mostrar las primeras filas para verificar la carga
print(df.head())

   producto idSecuencia  udsVenta  bolOpen  EnPromocion  anyomes  diasemana  \
0         1  2022-12-06  0.000000        1            0        2          2   
1         2  2022-12-06  1.098612        1            0        2          2   
2         3  2022-12-06  0.000000        1            0        2          2   
3         4  2022-12-06  0.000000        1            0        2          2   
4         5  2022-12-06  0.000000        1            0        2          2   

   semana  media_7_dias  media_30_dias  venta_lag_1  venta_lag_7  venta_lag_30  
0      49      2.120606       2.301086     2.833213     2.079442           0.0  
1      49      1.874395       2.042219     2.079442     1.791759           0.0  
2      49      1.844732       1.681282     2.302585     2.564949           0.0  
3      49      1.186639       1.540582     1.098612     0.000000           0.0  
4      49      1.571729       1.871285     2.302585     2.564949           0.0  


Preparación de los datos

In [3]:
# Definir las características (X) y la variable objetivo (y)
X = df[['producto', 'bolOpen', 'EnPromocion', 'diasemana', 'semana', 'media_7_dias', 'media_30_dias', 'venta_lag_1', 'venta_lag_7', 'venta_lag_30']].copy()  # Características
y = df['udsVenta']  # Variable objetivo

# Codificación de las variables categóricas
label_encoder = LabelEncoder()

# Transformar varialbes
X['producto'] = label_encoder.fit_transform(X['producto'])
X['bolOpen'] = label_encoder.fit_transform(X['bolOpen'])
X['EnPromocion'] = label_encoder.fit_transform(X['EnPromocion'])
X['diasemana'] = label_encoder.fit_transform(X['diasemana'])
X['semana'] = label_encoder.fit_transform(X['semana'])


In [4]:
# Crear una nueva columna para la clasificación (0 si no hay ventas, 1 si hay ventas)
df['ventas'] = np.where(df['udsVenta'] > 0, 1, 0)

In [5]:
# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dimensiones de X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

Dimensiones de X_train: (5608, 10), X_test: (1402, 10), y_train: (5608,), y_test: (1402,)


In [6]:
# Dividir los datos en dos conjuntos para clasificación (para identificar días con ventas)
X_train_clf, X_test_clf = X_train, X_test
y_train_clf = df.loc[X_train.index, 'ventas']
y_test_clf = df.loc[X_test.index, 'ventas']

### Clasificación

In [7]:
# Fase 1: Clasificación (XGBClassifier) para predecir si habrá ventas
clf = XGBClassifier(objective='binary:logistic', eval_metric='logloss', eta=0.1, max_depth=6, colsample_bytree=0.8, subsample=0.8, nthread=4)
clf.fit(X_train_clf, y_train_clf)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, nthread=4, ...)

In [8]:
# Predecir si habrá ventas en los días de prueba
y_pred_clf = clf.predict(X_test_clf)

### Regresión

In [9]:
# Fase 2: Regresión (XGBRegressor) solo para los días en los que hay ventas
#X_train_reg = X_train[y_train_clf == 1]  # Solo días con ventas
#y_train_reg = y_train[y_train_clf == 1]

#X_test_reg = X_test[y_pred_clf == 1]  # Solo días predichos con ventas
#y_test_reg = y_test[y_pred_clf == 1]

In [10]:
# Fase 2: Regresión (XGBRegressor) para predecir las unidades vendidas (tanto para días con ventas como sin ventas)
# Usamos todo el conjunto de datos para regresión
reg = XGBRegressor(objective='reg:squarederror', eval_metric='rmse', eta=0.1, max_depth=6, colsample_bytree=0.8, subsample=0.8, nthread=4)
reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric='rmse',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, nthread=4, ...)

In [11]:
# Predecir las ventas en el conjunto de prueba
y_pred_reg = reg.predict(X_test)

In [12]:
# Para los días sin ventas, el modelo de clasificación predice 0 (no hay ventas), por lo tanto ajustamos las predicciones
y_pred_reg_adjusted = y_pred_reg * y_pred_clf  # Multiplicamos por la predicción de clasificación para poner 0 en días sin ventas

In [13]:
# Evaluar el modelo de regresión utilizando RMSE, MSE y MAE
rmse = mean_squared_error(y_test, y_pred_reg_adjusted, squared=False)
mse = mean_squared_error(y_test, y_pred_reg_adjusted)
mae = mean_absolute_error(y_test, y_pred_reg_adjusted)
r2 = r2_score(y_test, y_pred_reg_adjusted)

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R2: {r2}")


RMSE: 0.7324191287759814
MSE: 0.5364377801969675
MAE: 0.5136424121281895
R2: 0.48337305180876766


In [14]:
# guardar modelos
clf.save_model('combinacion_xgb_01_classifier_model.json')
reg.save_model('combinacion_xgb_01_regressor_model.json')

En resumen, el modelo es aceptable, pero con margen para mejorar. Un R² de 0.4834 significa que el modelo explica aproximadamente el 48.34% de la variabilidad en las ventas.